In [1]:
from IPython.display import Image

# 15. 심층 합성곱 신경망으로 이미지 분류
* 1차원과 2차원의 합성곱 연산
* CNN 구조의 구성 요소
* 텐서플로를 사용하여 심층 합성곱 신경망 구현
* 일반화 성능 향상을 위한 데이터 증식
* 얼굴 이미지에서 성별을 예측하는 합성곱 신경망 구현

# 15.1 합성곱 신경망의 구성 요소

### 15.1.1 CNN과 특성 계층 학습
CNN과 같은 종류의 신경망은 원본 데이터에서 작업에 가장 유용한 특성을 자동으로 학습할 수 있다. 이런 이유로 CNN 층을 특성 추출기로 생각하기도 한다. (입력층 바로 다음에 있는) 층은 원본 데이터에서 저수준 특성을 추출한다. (종종 다층 퍼셉트론(MLP)과 같은 완전 연결 층으로 만드는) 뒤쪽의 층은 이런 특성을 사용하여 연속적인 타깃 값이나 클래스 레이블을 예측한다.

특정 종류의 다층 신경망과 특히 심층 합성곱 신경망은 각 층별로 저수준 특성을 연결하여 고수준 특성을 만듦으로써 소위 특성 계층을 구성한다. 예를 들어 이미지를 다룬다면 모서리나 동그라미 같은 저수준 특성이 앞쪽 층에서 추출된다. 이 특성들이 연결되어 고수준 특성을 형성한다. 이런 고수준 특성은 건물, 자동차, 강아지 같은 더 복잡한 모양을 형성할 수 있다.

CNN은 입력 이미지에서 **특성 맵**(feature map)을 만든다. 이 맵의 각 원소는 입력 이미지의 국부적인 픽셀 패치에서 유도된다.

In [2]:
# 언스플래시(Unsplash)에 있는 알렉산더 더머의 사진
Image(url='https://git.io/JL5O3', width=700)

이러한 국부적인 픽셀 패치를 국부 수용장이라고 말한다.

* CNN의 아이디어
    * 희소 연결: 특성 맵에 있는 하나의 원소는 작은 픽셀 패치 하나에만 연결
    * 파라미터 공유: 동일한 가중치가 입력 이미지의 모든 패치에 사용

CNN은 여러 개의 합성곱(conv) 층과 풀링(Pooling, P)이라고도 하는 서브샘플링(subsampling) 층으로 이루어져 있다. 마지막에는 하나 이상의 완전 연결(FC) 층이 따라온다. 완전 연결 층은 모든 입력 유닛 i가 모든 출력 유닛 j에 가중치 $w_{ij}$로 연결되어 있는 다층 퍼셉트론이다.

* 서브샘플링 층(풀링 층)
    * 학습되는 파라미터가 없다.
        * 풀링 층에 가중치나 절편 유닛이 없다.
* 합성곱이나 완전 연결 층은 훈련 도중 최적화되는 가중치와 절편을 가진다.    

### 15.1.2 이산 합성곱 수행
* 이산 합성곱(discrete convolution, 합성곱)
    * CNN의 기본 연산

* 1차원 이산 합성곱 연산 수행
    * 두 개의 벡터 x와 w에 대한 이산 합성곱은 y = x * w
        * x: 입력(신호)
        * w: 필터 또는 커널


$y=x*w$ -> $y[i] = \sum_{k=-∞}^{+∞}x[i-k]w[k]$

1. 인덱스 -∞부터 +∞까지의 합
    * 머신러닝 애플리케이션은 항상 유한한 특성 벡터를 다룬다.
        * ex, x가 0, 1, 2, ..., 8, 9 인덱스로 열 개의 특성을 가지고 있다면 -∞:-1과 10:+∞ 인덱스는 x의 범위 밖이다. 덧셈을 올바르게 하려면 x와 w가 0으로 채워져 있다고 가정해야 한다. 또한, 출력 벡터 y도 0으로 채워진 무한 크기가 된다.
        * 실제 상황에서는 유용하지 않기 때문에 유한한 개수의 0으로 x가 **패딩**된다.
    * **제로 패딩** 또는 **패딩**
        * p: 각 방향으로 추가된 패딩 수

In [3]:
# 1차원 패딩 예
Image(url='https://git.io/JL5On', width=700)

원본 입력 x와 필터 w가 각각 n개, m개의 원소를 가지고 있고 m <= n 이라고 가정. 패딩된 벡터 $x^p$ 크기는 n + 2p 이다.

$y = x * w$ -> $y[i] = \sum_{k=0}^{k=m-1}x^p[i+m-k]w[k]$

2. i+m-k로 x 인덱싱
    * x와 w가 다른 방향으로 인덱싱.
        * 하나가 반대 방향으로 인덱싱되는 계산은 패딩된 후 x 또는 w 벡터 중 하나를 뒤집어 계산하는 것과 동일.
        * 필터 w를 뒤집어서 회전된 필터 $w^r$를 얻었다고 가정.
        * 점곱 x[i:i+m]*$w^r$을 계산하면 y[i] 원소 하나가 얻어진다.
            * x[i:i+m]은 크기가 m인 x의 패치.
            * 이 연산은 모든 출력 원소를 얻기 위해 슬라이딩 윈도우 방식으로 반복

In [4]:
# 1차원 합성곱
Image(url='https://git.io/JL5O8', width=700)

패딩 크기는 0(p=0)이다. 회전된 필터 $w^r$은 2칸씩 이동한다. 이동하는 양은 **스트라이드**(stride)라고 하며, 합성곱의 하이퍼파라미터이다. 스트라이드는 입력 벡터의 크기보다 작은 양수값이어야 한다.

* 출력 특성 맵의 크기를 조절하기 위해 입력에 패딩
  * 풀(full) 패딩
    * 패딩 파라미터 p를 m-1로 설정.
    * 출력 크기를 증가시키기 때문에 합성곱 신경망 구조에서는 거의 사용되지 않는다.
  * 세임(same) 패딩
    * 파딩 파라미터 p는 입력과 출력 크기가 동일해야 하기 때문에 필터 그기에 따라 결정
    * 출력 크기가 입력 벡터 x와 같아야할 때 사용.
  * 밸리드(valid) 패딩
    * p=0
    * 패딩 없음
    

In [5]:
Image(url='https://git.io/JL5Ow', width=700)

합성곱 신경망에서 가장 많이 사용되는 패딩 방법은 **세임** 패딩이다. 

* 세임 패딩 장점: **벡터 크기를 유지**시킨다는 것이다.

컴퓨터 비전 분야의 이미지 관련 작이라면 입력 이미지의 높이와 너비가 유지된다. 이 때문에 네트워크 구조를 설계하기 쉽다.

* 밸리드 패딩 단점: 신경망에 층이 추가될수록 점진적으로 텐서 크기가 줄어든다.
    * 신경망 성능을 나쁘게 만들 수 있다.

실전에서는 세임 패딩으로 너비와 높이를 유지시키고 풀링에서 크기를 감소시킨다. 풀 패딩은 입력보다 출력 크기를 증가시키므로 경계 부분의 영향을 최소화하는 것이 중요한 신호 처리 애플리케이션에서 보통 사용된다.

* 합성곱 출력 크기 계산
    * 입력 벡터 위를 필터 w가 이동하는 전체 횟수
    * n: 입력 벡터 크기
    * m: 필터 크기
    * p: 패딩
    * s: 스트라이드
    
$$o = \frac{n + 2p - m}{s} + 1$$

$\frac{n + 2p - m}{s}$의 소수점 아래 값 버림.

In [6]:
import numpy as np
import tensorflow as tf

In [7]:
# 1차원 합성곱 계산
def conv1d(x, w, p=0, s=1):
    w_rot = np.array(w[::-1])
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_padded = np.concatenate(
            [zero_pad, x_padded, zero_pad])
    res = []
    for i in range(0, int((len(x_padded) - len(w_rot)) / s) + 1, s):
        res.append(np.sum(x_padded[i:i+w_rot.shape[0]] * w_rot))
    return np.array(res)

# 테스트
x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]

print('Conv1d 구현:',
      conv1d(x, w, p=2, s=1))

print('넘파이 결과:',
      np.convolve(x, w, mode='same')) 

Conv1d 구현: [ 5. 14. 16. 26. 24. 34. 19. 22.]
넘파이 결과: [ 5 14 16 26 24 34 19 22]


* 2D 이산 합성곱 수행


In [8]:
# 입력 행렬 크기 = 8 * 8
# 커널 크기 = 3 * 3
# 입력 행렬이 p=1로 제로 패딩. => 2D 합성곱은 8 * 8 크기의 출력을 만듬
Image(url='https://git.io/JL5OP', width=700)

In [9]:
# 패딩 p = (1, 1)
# 스트라이드 s = (2, 2)
# 입력 행렬 X 크기 = 3 * 3 -> 입력 행렬의 네 면에 0이 한 줄씩 추가(패딩)되어 X_padded 크기 = 5 * 5
# 커널 행렬 W 크기 = 3 * 3
Image(url='https://git.io/JL5OD', width=600)

In [10]:
# W 뒤집기 -> W[::-1, ::-1]
# X_padded를 따라 슬라이딩 윈도우처럼 역전된 필터를 이동하면서 원소별 곱의 합 계산
# 결과 Y는 2 * 2 크기 행렬
Image(url='https://git.io/JL5OS', width=800)

In [11]:
# 2D 합성곱
import scipy.signal # 2D 합성곱을 계산할 수 있는 scipy.signal.convolve2d 함수 제공

def conv2d(X, W, p=(0, 0), s=(1, 1)):
    W_rot = np.array(W)[::-1, ::-1]
    X_orig = np.array(X)
    n1 = X_orig.shape[0] + 2 * p[0]
    n2 = X_orig.shape[1] + 2 * p[1]
    X_padded = np.zeros(shape=(n1, n2))
    X_padded[p[0]:p[0]+X_orig.shape[0],
             p[1]:p[1]+X_orig.shape[1]] = X_orig
    res = []
    for i in range(0, int((X_padded.shape[0] - W_rot.shape[0]) / s[0])+1, s[0]):
        res.append([])
        for j in range(0, int((X_padded.shape[1] - W_rot.shape[1]) / s[1])+1, s[1]):
            X_sub = X_padded[i:i+W_rot.shape[0], j:j+W_rot.shape[1]]
            res[-1].append(np.sum(X_sub * W_rot))
    return np.array(res)

X = [[1, 3, 2, 4], [5, 6, 1, 3], [1, 2, 0, 2], [3, 4, 3, 2]]
W = [[1, 0, 3], [1, 2, 1], [0, 1, 1]]

print('Conv2d 구현:\n',
    conv2d(X, W, p=(1, 1), s=(1, 1)))


print('싸이파이 결과:\n',
    scipy.signal.convolve2d(X, W, mode='same'))

Conv2d 구현:
 [[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]
싸이파이 결과:
 [[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]


### 15.1.3 서브샘플링
* 전형적인 두 종류의 풀링 연산으로 합성곱 신경망에 적용된다.
    * 최대 풀링(max-pooling): 이웃한 픽셀에서 최댓값 계산
    * 평균 풀링(mean-pooling 또는 average-pooling): 이웃한 픽셀에서 픽셀 평균 계산

* 풀링 층은 보통 $P_{n1*n2}$ 로 표시
    * 아래 첨자는 최댓값과 평균 연산이 수행되는 이웃한 픽셀 크기
        * 차원별로 인접 픽셀 개수
    * 이웃 픽셀 개수를 풀링 크기라고 한다.

In [12]:
# 최대 풀링과 평균 풀링
Image(url='https://git.io/JL5OH', width=700)

* 풀링의 장점
    1. 풀링(최대 풀링)은 **지역 불변성**을 만든다. 국부적인 작은 변화가 최대 풀링의 결과를 바꾸지 못한다는 의미. 결국 입력 데이터에 있는 **잡음에 조금 더 안정**적인 특성을 생성한다.
    2. 풀링은 특성 크기를 줄이므로 계산 효율성을 높인다. 또 특성 개수가 줄어들면 과대적합도 감소된다.

* 겹치는 풀링 vs 겹치지 않는 풀링
    * 전통적으로 풀링은 겹치지 않는다고 가정한다.
        * 풀링이 겹치지 않도록 수행되기 때문에 일반적으로 스트라이드 크기를 풀링 크기와 같게 설정한다.
        * ex, 겹치지 않는 풀링 층 P_n1*n2의 스트라이드 s는 (n1, n2)이다.
    * 스트라이드가 풀링 크기보다 작으면 겹쳐서 풀링이 일어난다.

# 15.2 기본 구성 요소를 사용하여 심층 합성곱 신경망 구성
* 합성곱 연산 Z = W * X + b
    * X는 높이 * 너비의 픽셀을 나타내는 행렬
    * 은닉 유닛의 활성화 출력 A = Φ(Z)를 얻기 위해 활성화 함수에 입력으로 전달.
        * Φ: 활성화 함수

### 15.2.1 여러 개의 입력 또는 컬러 채널 다루기
합성곱 층의 입력 샘플에는 N1 * N2 차원(ex, 이미지의 높이와 너비 픽셀)인 하나의 2D 배열 또는 행렬이 포함될 수 있다. 이런 N1 * N2 행렬을 **채널**이라고 한다.

합성곱 층은 랭크 3 텐서를 입력으로 한다. 즉 , $X_{N_{1}*N_{2}*C_{in}}$을 사용. 여기서 $C_{m}$은 입력 채널 크기이다. RGB 모드인 컬러일 경우 $C_{in}$=3이다. 그레이스케일(grayscale)인 흑백일 경우 $C_{in}$=1이다.

In [13]:
# 이미지 다운로드
!wget https://git.io/JL5Ob -O example-image.png

--2022-02-10 04:49:41--  https://git.io/JL5Ob
Resolving git.io (git.io)... 52.204.242.176, 18.205.36.100, 54.157.58.70, ...
Connecting to git.io (git.io)|52.204.242.176|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch15/example-image.png [following]
--2022-02-10 04:49:42--  https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch15/example-image.png
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch15/example-image.png [following]
--2022-02-10 04:49:43--  https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch15/example-image.png
Resolving raw.githubusercontent.com (raw.githubuserconte

이미지를 다룰 때 메모리 절약을 위해 16비트, 32비트, 64비트 정수 타입 대신 uint8(부호 없는 8비트 정수) 데이터 타입의 넘파이 배열로 이미지를 읽을 수 있다.

부호 없는 8비트 정수는 [0, 255] 사이 값을 저장할 수 있는데 RGB 이미지의 픽셀 정보도 같은 범위이므로 충분하다.

In [14]:
# 이미지 파일 읽기 - tensorflow
import tensorflow as tf

img_raw = tf.io.read_file('example-image.png')
img = tf.image.decode_image(img_raw)
print('이미지 크기:', img.shape)
print('채널 개수:', img.shape[2])
print('이미지 데이터 타입:', img.dtype)
print(img[100:102, 100:102, :])

이미지 크기: (252, 221, 3)
채널 개수: 3
이미지 데이터 타입: <dtype: 'uint8'>
tf.Tensor(
[[[179 134 110]
  [182 136 112]]

 [[180 135 111]
  [182 137 113]]], shape=(2, 2, 3), dtype=uint8)


In [15]:
# 이미지 파일 읽기 - imageio
import imageio

img = imageio.imread('example-image.png')
print('이미지 크기:', img.shape)
print('채널 개수:', img.shape[2])
print('이미지 데이터 타입:', img.dtype)
print(img[100:102, 100:102, :])

이미지 크기: (252, 221, 3)
채널 개수: 3
이미지 데이터 타입: uint8
[[[179 134 110]
  [182 136 112]]

 [[180 135 111]
  [182 137 113]]]


#### CNN 입력을 위한 흑백 이미지의 랭크

In [16]:
!wget https://git.io/JL5Op -O example-image-gray.png

--2022-02-10 04:49:43--  https://git.io/JL5Op
Resolving git.io (git.io)... 52.204.242.176, 18.205.36.100, 54.157.58.70, ...
Connecting to git.io (git.io)|52.204.242.176|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch15/example-image-gray.png [following]
--2022-02-10 04:49:44--  https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch15/example-image-gray.png
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch15/example-image-gray.png [following]
--2022-02-10 04:49:45--  https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch15/example-image-gray.png
Resolving raw.githubusercontent.com 

In [17]:
img_raw = tf.io.read_file('example-image-gray.png')
img = tf.image.decode_image(img_raw)
tf.print('랭크:', tf.rank(img))
tf.print('크기:', img.shape)

랭크: 3
크기: TensorShape([252, 221, 1])


In [18]:
img = imageio.imread('example-image-gray.png')
tf.print('랭크:', tf.rank(img))
tf.print('크기:', img.shape)

img_reshaped = tf.reshape(img, (img.shape[0], img.shape[1], 1))
tf.print('새로운 크기:', img_reshaped.shape)

랭크: 2
크기: (252, 221)
새로운 크기: TensorShape([252, 221, 1])


보통 CNN의 합성곱 층은 하나 이상의 특성 맵(h)을 만든다. 여러 개의 특성 맵을 사용하면 커널 텐서는 height * width * $C_{in}$ * $C_{out}$으로 4차원이 된다. 높이와 너비는 커널의 크기이고 $C_{in}$은 입력 채널의 개수, $C_{out}$은 출력 특성 맵의 개수이다.

### 15.2.2 드롭아웃으로 신경망 규제
* 네트워크의 크기에 대한 문제를 해결하는 방법
    1. 훈련 데이터셋에서 잘 동작하도록 비교적 큰 용량의 네트워크를 구축(실제로 필요한 것보다 좀 더 큰 용량을 선택)
    2. 과대적합을 막기 위해 한 개 이상의 규제 방법을 적용하여 별도의 테스트 데이터셋 같은 새로운 데이터에서 일반화 성능 높임



In [19]:
# L2 규제
from tensorflow import keras
conv_layer = keras.layers.Conv2D(
    filters=16,
    kernel_size=(3, 3),
    kernel_regularizer=keras.regularizers.l2(0.001))
fc_layer = keras.layers.Dense(
    units=16,
    kernel_regularizer=keras.regularizers.l2(0.001))

* 드롭아웃(dropout)
    * 보통 뒤쪽 층의 은닉 유닛에 적용.
    * 신경망을 훈련하는 동안 반복마다 은닉 유닛의 일부가 확률 P_drop만큼 랜덤하게 드롭아웃 된다.
    * 드롭아웃 확률은 사용자가 지정.
        * 가장 많이 사용하는 값은 p=0.5이다.
    * 입력 뉴런의 일부를 드롭아웃할 때 없어진(드롭아웃된) 뉴런을 보상하기 위해 남은 뉴런에 연결된 가중치 값을 크게 한다.
    * 랜덤한 드롭아웃의 영향으로 네트워크는 데이터에서 여분의 표현을 학습한다.
        * 네트워크가 일부 은닉 유닛의 활성화 값에 의존할 수 없다.
            * 훈련과정에서 언제든지 은닉 유닛이 꺼질 수 있기 때문
        * 네트워크가 데이터에서 더 일반적으로 안정적인 패턴을 학습하게 만든다.
    * 랜덤한 드롭아웃은 과대적합을 효과적으로 방지.

In [20]:
# 드롭아웃 예
Image(url='https://git.io/JL5Oh', width=700)

훈련 단계에서만 유닛이 랜덤하게 꺼진다는 것이 중요. 평가(추론) 단계에서는 모든 유닛이 활성화되어야 한다.(즉, P_drop = 0, P_keep = 1)

훈련과 예측 단계의 전체 활성화 값의 스케일을 맞추기 위해 활성화된 뉴런 출력이 적절히 조정되어야 한다.(ex, 훈련 시 드롭아웃 p=0.5일 때 테스트 시 활성 출력을 절반으로 낮춘다)

실전에서 예측을 만들 때 활성화 값의 출력을 조정하는 것은 불편하기 때문에 텐서플로나 다른 라이브러리들은 훈련 단계의 활성화를 조정한다. 이런 방법을 **역 드롭아웃**이라 한다.

드롭아웃은 앙상블 모델의 기하 평균이 훈련 과정에서 샘플링된 마지막(또는 최종) 모델의 예측에 1 / (1-p)를 곱하는 것으로 근사할 수 있다.

### 15.2.3 분류를 위한 손실 함수
* 렐루와 같은 몇몇 활성화 함수는 모델에 비선형성을 더하기 위해 신경망의 중간(은닉)층에 주로 사용.
    * (이진 분류일 경우) 시그모이드와 (다중 분류일 경우) 소프트맥스 같은 함수는 마지막 (출력)층에 추가되어 클래스 소속 롹률을 출력
    * 시그모이드와 소프트맥스 활성화 함수가 출력층에 포함되지 않으면 모델은 클래스 소속 확률 대신 로짓을 계산할 것이다.

* 분류 모델
 * **`BinaryCrossentropy()`**
   * `from_logits=False` 
   * `from_logits=True`

 * **`CategoricalCrossentropy()`**
   * `from_logits=False`
   * `from_logits=True`
   
 * **`SparseCategoricalCrossentropy()`**
   * `from_logits=False`
   * `from_logits=True`

In [21]:
# 분류 문제에 사용하는 손실 함수
Image(url='https://git.io/JL53f', width=800)

클래스 소속 확률이 아니라 **로짓으로 크로스 엔트로피 손실을 계산하는 것이 수치상의 안정성 때문에 일반적으로 선호**된다. 손실 함수의 입력으로 로짓을 사용하고 `from_logits=True`로 지정하면 해당하는 텐서플로 함수는 훨씬 효율적인 구현을 사용하여 손실과 가중치에 대한 손실의 도함수를 계산한다. 로짓이 입력으로 제공되면 일부 수학 항을 소거할 수 있어 굳이 계산하지 않아도 되기 때문이다.

In [22]:
# 로짓이나 클래스 소속 확률이 손실 함수의 입력으로 주어졌을 때 세 개의 손실 함수를 사용하는 방법
import tensorflow_datasets as tfds

# 이진 크로스 엔트로피
bce_probas = tf.keras.losses.BinaryCrossentropy(from_logits=False)
bce_logits = tf.keras.losses.BinaryCrossentropy(from_logits=True)

logits = tf.constant([0.8])
probas = tf.keras.activations.sigmoid(logits)

print(
    'BCE (확률): {:.4f}'.format(bce_probas(y_true=[1], y_pred=probas)),
    '(로짓): {:.4f}'.format(bce_logits(y_true=[1], y_pred=logits)))

BCE (확률): 0.3711 (로짓): 0.3711


In [23]:
# 범주형 크로스 엔트로피
cce_probas = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
cce_logits = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

logits = tf.constant([[1.5, 0.8, 2.1]])
probas = tf.keras.activations.softmax(logits)

tf.print(
    'CCE (확률): {:.4f}'.format(cce_probas(y_true=[[0, 0, 1]], y_pred=probas)),
    '(로짓): {:.4f}'.format(cce_logits(y_true=[[0, 0, 1]], y_pred=logits)))

CCE (확률): 0.5996 (로짓): 0.5996


In [24]:
# 희소 범주형 크로스 엔트로피
sp_cce_probas = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
sp_cce_logits = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

tf.print(
    'Sparse CCE (확률): {:.4f}'.format(sp_cce_probas(y_true=[2], y_pred=probas)),
    '(로짓): {:.4f}'.format(sp_cce_logits(y_true=[2], y_pred=logits)))

Sparse CCE (확률): 0.5996 (로짓): 0.5996


가끔 샘플마다 두 개의 출력을 얻어 각 클래스에 대한 확률 (P[class=0] 대 P[calss=1])로 해석되기도 한다. 이런 경우 로지스틱 시그모이드 대신 소프르맥수 함수를 사용하여 출력을 정규화(즉, 합이 1이 되로곡)하는 것이 좋다. 이때는 범주형 크로스 엔트로피가 손실 함수로 적절.